In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/cbarnes5/DATA606CapstoneProject/main/finalcsv.csv'
df = pd.read_csv(url, index_col = 0)

In [2]:
df

,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,HOA/MONTH,LATITUDE,LONGITUDE
0,2024-04-15,Condo/Co-op,1391 Pennsylvania Ave SE #354,Washington,DC,20003.0,295000.0,0.0,1.0,Old City 1,451.0,NaN,2007.0,310.0,38.880206,-76.987612
1,2024-05-10,Townhouse,1346 K St SE,Washington,DC,20003.0,560000.0,3.0,1.0,Old City 1,1400.0,1752.0,1926.0,0.0,38.878566,-76.986884
2,2024-03-28,Condo/Co-op,1391 Pennsylvania Ave SE #311,Washington,DC,20003.0,695000.0,2.0,2.0,Old City 1,1234.0,NaN,2007.0,831.0,38.880206,-76.987612
3,2024-03-22,Townhouse,1507 Freedom Way SE,Washington,DC,20003.0,785000.0,3.0,3.5,HILL EAST,1766.0,NaN,1979.0,0.0,38.879773,-76.982862
4,2024-04-15,Condo/Co-op,1391 Pennsylvania Ave SE #426,Washington,DC,20003.0,425000.0,1.0,1.0,Old City 1,684.0,NaN,2007.0,475.0,38.880206,-76.987612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,2024-04-26,Single Family Residential,2706 Valley Dr,Alexandria,VA,22302.0,1625000.0,5.0,4.5,BRADDOCK HEIGHTS,3600.0,7050.0,1940.0,0.0,38.828810,-77.079460
2425,2024-05-16,Condo/Co-op,3212 Wellington Rd #103,Alexandria,VA,22302.0,429000.0,2.0,1.0,PARKFAIRFAX,900.0,NaN,1941.0,542.0,38.834207,-77.077879
2426,2024-04-12,Townhouse,1724 Preston Rd,Alexandria,VA,22302.0,422000.0,2.0,1.0,PARKFAIRFAX,930.0,NaN,1941.0,561.0,38.835865,-77.082432
2427,2024-04-26,Condo/Co-op,1606 Ripon Pl,Alexandria,VA,22302.0,380000.0,2.0,1.0,PARKFAIRFAX,900.0,NaN,1941.0,495.0,38.834562,-77.082253


First we want to figure out how to handle non-numeric columns. We would like to consider the property type, as well as the area the house is in. For area, we can consider the city, zip code, or neighborhood (the location column). Since these are obviously highly correlated, we would only like to consider one of them

In [7]:
len(df['PROPERTY TYPE'].unique()) #Nice for one-hot encoding

4

In [6]:
len(df['ZIP OR POSTAL CODE'].unique())

47

In [17]:
len(df['LOCATION'].unique())

423

In [18]:
len(df['CITY'].unique()) #Looks like some other cities snuck into our data

14

In [20]:
df['CITY'].value_counts() #Could potentially make an "other" tag for cities not in our main 3, or omit this data

CITY
Washington         1456
Arlington           639
Alexandria          282
Falls Church          9
Silver Spring         6
Oxon Hill             4
Hyattsville           4
Chevy Chase           3
Suitland              2
Capitol Heights       2
Takoma Park           2
Temple Hills          1
Mount Rainier         1
Mclean                1
Name: count, dtype: int64

In [9]:
pd.set_option('display.max_rows', None)
print(df['LOCATION'].value_counts())
pd.reset_option('display.max_rows')

LOCATION
Old City 1                                  163
Old City 2                                  120
Columbia Heights                             80
Petworth                                     59
Brookland                                    58
Georgetown                                   55
Chevy Chase                                  51
Congress Heights                             40
Capitol Hill                                 39
Cleveland Park                               37
Trinidad                                     37
Central                                      36
Forest Hills                                 35
Brightwood                                   31
Deanwood                                     30
Observatory Circle                           27
Wesley Heights                               27
Randle Heights                               25
Fort Dupont Park                             24
American University                          23
Riggs Park                     

In [10]:
pd.set_option('display.max_rows', None)
print(df['ZIP OR POSTAL CODE'].value_counts())
pd.reset_option('display.max_rows')

ZIP OR POSTAL CODE
20002.0    171
20011.0    153
22314.0    146
20007.0    134
22201.0    129
22204.0    120
20016.0    111
20003.0    107
20009.0     85
20008.0     81
22207.0     79
20019.0     78
22209.0     70
22203.0     68
20020.0     62
22206.0     59
22205.0     57
20017.0     55
20010.0     54
20001.0     53
20015.0     52
22301.0     50
20037.0     47
22305.0     46
22202.0     46
20032.0     42
20018.0     42
22302.0     40
20024.0     34
20012.0     33
20005.0     33
20036.0     28
22213.0     11
22044.0      7
20910.0      6
20745.0      4
20815.0      3
20782.0      3
20912.0      2
22041.0      2
20746.0      2
20743.0      2
20783.0      1
22101.0      1
20006.0      1
20712.0      1
20748.0      1
Name: count, dtype: int64


We obviously have too may neighborhoods and zip codes to utilize one-hot encoding. We could incorporate target encoding (ie, make a column with the mean of sales price for the location/zip code a house is in) instead.

However there is the concern that we have many locations that only have a tiny amount of houses sampled, which brings up the concern of overfitting. Also the way that "locations" are entered in our data are a bit inconsistent (for example, there is an "Old City 1" label but also an "OLD CITY #1" label, "ECKINGTON" and "Eckington", "OLD TOWN" and "OLD TOWNE", "VIRGINIA HEIGHTS" and "VIRGINIA HEIGHTS ETC", and so on. Pinpointing and addressing every kink for this column of 423 unique values does not seem viable.

This leads us to prefer to use zip code as a variable of interest. I am a bit concerned that we might lose information, as two different neighborhoods in the same zip code could have a high discrepancy in mean sales price. Consider the following example



In [44]:
common_zip = df.loc[df['ZIP OR POSTAL CODE'] == 20002]
common_zip['LOCATION'].value_counts()

LOCATION
Old City 1           92
Trinidad             37
Eckington            12
Capitol Hill          7
KINGMAN PARK          5
CARVER LANGSTON       5
Brookland             4
ECKINGTON             3
Brentwood             1
H STREET CORRIDOR     1
OLD CITY #1           1
NOMA                  1
Woodridge             1
BROOKLAND             1
Name: count, dtype: int64

In [46]:
common_zip.groupby('LOCATION')['PRICE'].mean()

LOCATION
BROOKLAND            7.950000e+05
Brentwood            3.150000e+05
Brookland            6.367250e+05
CARVER LANGSTON      3.409200e+05
Capitol Hill         1.651571e+06
ECKINGTON            6.629333e+05
Eckington            9.230458e+05
H STREET CORRIDOR    3.199000e+05
KINGMAN PARK         6.940000e+05
NOMA                 1.100000e+06
OLD CITY #1          3.370000e+05
Old City 1           9.653476e+05
Trinidad             5.800614e+05
Woodridge            8.070000e+05
Name: PRICE, dtype: float64

There is still a notable variance among neighborhoods in the same zip code. Makes us think ignoring the neighborhood column would be a crucial loss of information. However, it is a messy column to just target encode the entire thing. Is there a method where we target encode for neighborhoods with a certain number of houses in the sample? Could potentially use the neighborhood sale price mean for neighborhoods with >= 30 houses, then resort to a city/zip code mean for neighborhoods with inconclusive data. Will run through professor before encoding